# Aeolian Alexanders: Economic, Material, and Social Networks in Antiquity

Supported by a NEH - Mellon Digital Publication Grant <img src="img/neh_seal.jpg" alt="NEH Seal" style="width: 200px;"/>

# Introduction

## Headers

In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import pandas as pd
import psycopg2
import sqlalchemy
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.font_manager as font_manager
import matplotlib
import argparse
import numpy as np
import matplotlib.font_manager as font_manager
import json
from matplotlib.ticker import FormatStrFormatter
import folium
from folium import plugins


In [6]:
import geopandas
import networkx as nx
import community
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import geopandas as gpd
import contextily as ctx
from pygraphviz import *
from networkx.algorithms import bipartite
from folium.features import DivIcon
from natsort import natsorted
import sqlite3

from IPython.display import display

ImportError: dlopen(/opt/anaconda3/lib/python3.7/site-packages/fiona/ogrext.cpython-37m-darwin.so, 2): Library not loaded: @rpath/libpoppler.76.dylib
  Referenced from: /opt/anaconda3/lib/libgdal.20.dylib
  Reason: image not found

In [ ]:
from sqlalchemy import create_engine 
cnx = create_engine('sqlite:///aeolian_alexanders.db')

In [ ]:
# data structures used in the application. You can define any event and year soan following these models

third_syrian_war = {
    "start":-246,
    "end":-241
}

war_of_antiochos = {
    "start":-192,
    "end":-188
}

# In this case the Attalids are the focus of the study, so the dates of the monarchs are important
attalid_rulers = []

Philetaerus = {'start_date': -282, 'end_date':-263}
attalid_rulers.append(Philetaerus)
EumenesI = {'start_date': -263, 'end_date':-241}
attalid_rulers.append(EumenesI)
AttalusI = {'start_date': -241, 'end_date':-197}
attalid_rulers.append(AttalusI)
EumenesII = {'start_date': -197, 'end_date':-160}
attalid_rulers.append(EumenesII)
AttalusII = {'start_date': -160, 'end_date':-138}
attalid_rulers.append(AttalusII)
AttalusIII = {'start_date': -138, 'end_date':-133}
attalid_rulers.append(AttalusIII)

#Polities are defined by their Pleiades IDs and whatever title you wish to give them

polities = []
polity_1 = {'title': 'Attalid_Kingdom','pid':550812}
polities.append(polity_1)
polity_2 = {'title': 'Seleucid_Kingdom','pid':45635759}
polities.append(polity_2)
polity_3 = {'title': 'Ptoemaic_Kingdom','pid':463803480}
polities.append(polity_3)
polity_4 = {'title': 'Lysimachus_Kingdom','pid':501458}
polities.append(polity_4)

# mints are defined by their Pleiades IDs; this could easily be changed to Nomisma IDs depending on your data setup
mint = {
    
    'temnos': '550908',
    'myrina': '550756',
    'kyme': '550506'
}

# attribution for the map
attr = """
Tiles &copy; <a href='http://mapbox.com/' target='_blank'>MapBox</a> |
 Data &copy; <a href='http://www.openstreetmap.org/' target='_blank'>OpenStreetMap</a> and contributors, CC-BY-SA |
 Tiles and Data &copy; 2020 <a href='http://www.awmc.unc.edu' target='_blank'>AWMC</a>
 <a href='http://creativecommons.org/licenses/by-nc/3.0/deed.en_US' target='_blank'>CC-BY-NC 3.0</a>
"""

## Simple Utilities

In [ ]:
def linReScale (oldmax, oldmin, newmax, newmin, oldnumber):
    newnumber = (oldnumber - oldmin) / (oldmax - oldmin) * (newmax - newmin) + newmin
    return newnumber

def makeNetColor (row, my_colors):
    netColor = matplotlib.colors.to_hex(my_colors(row['partition']))
    #netColor = my_colors(row['partition'])
    return netColor

def makePleiadesGeoDataFromList(inputfile, id_column, outputname):
    pleaidesdf = pd.DataFrame()
    pleaidesdf = pd.read_csv(inputfile)  
    pleiadesIdList = pleaidesdf['pid'].tolist()
    sql = 'SELECT id, title, geom from places where ' + ' or '.join(('id = ' + str(n) for n in pleiadesIdList))
    pleiadesGeodf = gpd.GeoDataFrame.from_postgis(sql, cnx, geom_col='geom' )
    pleiadesGeodf.to_file(outputname, driver="GeoJSON")


## Functions

In [ ]:
### createmapvis makes our basic map
def createmapvis():
    
    m = folium.Map(
        location=[40.58058, 36.29883], 
        zoom_start=5,
        zoom_control=False,
        tiles='https://api.mapbox.com/v4/isawnyu.map-knmctlkh/{z}/{x}/{y}.png?access_token=pk.eyJ1IjoiaXNhd255dSIsImEiOiJja2FoNTRlMXIwOXYzMnpsbGJldGhyMjFqIn0.aDYskzcbh5inAn5JISgLyQ',
        API_key='pk.eyJ1IjoiaXNhd255dSIsImEiOiJja2FoNTRlMXIwOXYzMnpsbGJldGhyMjFqIn0.aDYskzcbh5inAn5JISgLyQ',
        name='AWMC Base',
        attr= attr)
    
    return m



#This produces a "traditional" coin catalog. This can be sorted if desired.
def coinCatalogWorking (mint):
    sql = """
    SELECT coinid AS id,
    typeid AS type,
    obvdie AS obverse,
    revdie AS reverse,
    weight,
    rotation,
    size,
    title
    FROM all_coins
    where mint = '{mint}' AND obvdie !='' and revdie !=''
    ORDER BY type, obverse, reverse
    """.format(mint = mint)
    
    dfCoinCat = pd.read_sql_query(sql, cnx)
    dfCoinCat["weight"] = pd.to_numeric(dfCoinCat["weight"])

    return dfCoinCat

#This function takes a mint, then creates the necessary .json files to put it into a d3js visualization using
#our html templates

def makeHtmlJson (mint):
    revObvDf = pd.DataFrame()
    obvTypeDf = pd.DataFrame()
    revTypeDf = pd.DataFrame()
    coinTypeDf = pd.DataFrame()
    coinObvDf = pd.DataFrame()
    coinRevDf = pd.DataFrame()
    
    coinNodesDf = pd.DataFrame()
    
    revObvDf = coinEdges('revdie', 'obvdie', mint)
    obvTypeDf = coinEdges('obvdie', 'typeid', mint)
    revTypeDf = coinEdges('revdie', 'typeid', mint)
    coinTypeDf = coinEdges('id', 'typeid', mint)
    coinObvDf = coinEdges('id', 'obvdie', mint)
    coinRevDf = coinEdges('id', 'revdie', mint)
    
    coinNodesDf = coinNodes(mint)
    
    finalJson = '{"nodes":' + coinNodesDf.to_json(orient='records') + ','
    finalJson = finalJson + '"type_links":' + coinTypeDf.to_json(orient='records') + ','
    finalJson = finalJson + '"coin_obverse_links":' + coinObvDf.to_json(orient='records') + ','
    finalJson = finalJson + '"coin_reverse_links":' + coinRevDf.to_json(orient='records') + ','
    finalJson = finalJson + '"coin_links":' + revObvDf.to_json(orient='records') + ','
    finalJson = finalJson + '"obverse_type_links":' + obvTypeDf.to_json(orient='records') + ','
    finalJson = finalJson + '"reverse_type_links":' + revTypeDf.to_json(orient='records') + '}'
    
    return finalJson

#This is a helper function for makeHtmlJson
def coinEdges(source, target, mint):
    sql = """
    SELECT {source} AS source, {target} AS target, COUNT ({source}) AS weight
    FROM all_coins
    where mint = '{mint}' AND obvdie !='' and revdie !=''
    GROUP BY source, target
    ORDER BY source
    """.format(mint = mint, source = source, target = target)
    
    dfObvRev = pd.read_sql_query(sql, cnx)
    
    return dfObvRev

#This is a helper function for makeHtmlJson
def coinNodes(mint):
    dfcoinCat = pd.DataFrame()
    dfobvCat = pd.DataFrame()
    dftypeCat = pd.DataFrame()

    projectUri = ''
    sql = """
    SELECT id, title, concat('{projectUri}', id) AS uri, 'coin' AS kind FROM all_coins 
    where mint = '{mint}' AND obvdie !='' and revdie !=''   
    """.format(mint = mint, projectUri = projectUri)
    dfcoinCat = pd.read_sql_query(sql, cnx)
    
    sql = """
    SELECT DISTINCT obvdie AS id, obvdie AS title, concat('{projectUri}', obvdie) AS uri, 'obverse' AS kind FROM all_coins 
    where mint = '{mint}' AND obvdie !='' and revdie !=''   
    """.format(mint = mint, projectUri = projectUri)
    
    dfobvCat = pd.read_sql_query(sql, cnx)
    dfcoinCat = dfcoinCat.append(dfobvCat, ignore_index = True) 
    
    sql = """
    SELECT DISTINCT revdie AS id, obvdie AS title, concat('{projectUri}', revdie) AS uri, 'reverse' AS kind FROM all_coins 
    where mint = '{mint}' AND obvdie !='' and revdie !=''   
    """.format(mint = mint, projectUri = projectUri)
    
    dfrevCat = pd.read_sql_query(sql, cnx)
    dfcoinCat = dfcoinCat.append(dfrevCat, ignore_index = True) 

    sql = """
    SELECT DISTINCT typeid AS id, typeid AS title, concat('{projectUri}', typeid) AS uri, 'type' AS kind FROM all_coins 
    where mint = '{mint}' AND obvdie !='' and revdie !=''   
    """.format(mint = mint, projectUri = projectUri)

    dftypeCat = pd.read_sql_query(sql, cnx)
    dfcoinCat = dfcoinCat.append(dftypeCat, ignore_index = True)
    
    return dfcoinCat

#this function displays hoards associated with a chosen mint in a catalog. 
def hoardsDisplay (mint, startdate, enddate, buffer):
    mintUri = 'https://pleiades.stoa.org/places/' + mint
    sd = startdate - buffer;
    ed = enddate - buffer;
    
    sql = """
        SELECT aa_hoards.hoard_id as id,
        MAX(aa_hoards.title) AS title,
        SUM(aa_mints.count) AS count,
        MAX(aa_mints.denomination) AS denomination,
        MAX(aa_mints.type) AS type,
        MAX(aa_hoards.b_start_date) AS burial_start,
        MAX(aa_hoards.b_end_date) AS burial_end,
        MAX(aa_hoards.contents) AS contents,
        MAX(aa_hoards.ex_start_date) AS discovered_at,
        MAX(aa_hoards.location_uri) AS location_uri
        FROM
        aa_mints
        LEFT JOIN
        aa_hoards
        ON
        aa_mints.hoard = aa_hoards.hoard_id
        WHERE 
        CAST(b_start_date as decimal) >= {sd} and CAST(b_end_date as decimal) <= {ed}
        AND
        aa_hoards.hoard_id NOT IN (SELECT cross_reference FROM aa_parent_child)
        AND
        aa_hoards.hoard_id IN (SELECT aa_mints.hoard from aa_mints WHERE aa_mints.mint_uri = '{mintUri}')
        AND
        aa_mints.mint_uri = '{mintUri}'
        GROUP BY aa_hoards.hoard_id, aa_mints.mint_uri
        ORDER BY burial_start; 
    """.format(sd = sd, ed = ed, mintUri=mintUri)
    dfHoardCat = pd.read_sql_query(sql, cnx)
    
    return dfHoardCat

#This function maps hoards which are associated with a selected mint
def mintHoardMapper (hoarddf, nodeAttribute, mint, nodeMinSize, nodeMaxSize, m):
    
    sql = """
    SELECT aa_locations.id AS location_uri, aa_locations.lat, aa_locations.lon FROM aa_locations;
    """
    locdf = pd.DataFrame()
    locdf = pd.read_sql_query(sql, cnx)
    
    finaldf = hoarddf[(hoarddf['location_uri']!="null")]
    finaldf = finaldf[(finaldf['location_uri']!="")]
    finaldf = finaldf.merge(locdf, how='left')
    
    finaldf['lat'] = pd.to_numeric(finaldf['lat'])
    finaldf['lon'] = pd.to_numeric(finaldf['lon'])
    finaldf['count'] = pd.to_numeric(finaldf['count'])
    
    oldmax = hoarddf[nodeAttribute].max()
    oldmin = hoarddf[nodeAttribute].min()
    
    finaldf['size'] = linReScale(oldmax, oldmin, nodeMaxSize, nodeMinSize, finaldf['count'])

    
    for index, row in finaldf.iterrows():
        if float(row['lat']) < 9999999:
            popupText = '{title}'.format(title = row['title'])
            folium.CircleMarker(location = [row['lat'], row['lon']],
                                popup = popupText,
                                radius = row['size'],
                                fill_color='black', 
                                color = 'black', 
                                fill_opacity=0.7,
                                weight = .5
                               ).add_to(m)
    # now to put in the mint
    
    sql = """
    SELECT places.id, places.title, places.reprlat, places.reprlong
    FROM places
    WHERE id = {mint};
    """.format(mint = mint)
    
    mintdf = pd.DataFrame()
    mintdf = pd.read_sql_query(sql, cnx)
    title = mintdf['title']
    folium.Marker([mintdf['reprlat'], mintdf['reprlong']], popup='{title}'.format(title = title)).add_to(m)
    
    return m

#this displays hoards as an ego network around a chosen mint
def hoardsEgoDisplay (mint, startdate, enddate, buffer):
    sd = startdate - buffer;
    ed = enddate - buffer;
    mintUri = 'https://pleiades.stoa.org/places/' + mint
    sql = """
        SELECT aa_hoards.hoard_id,
        MAX(aa_hoards.title) AS hoard_title,
        MAX(aa_mints.mint) AS mint_title,
        mint_uri,
        SUM(aa_mints.count) AS count,
        MAX(aa_hoards.location_uri) AS location_uri
        FROM
        aa_mints
        LEFT JOIN
        aa_hoards
        ON
        aa_mints.hoard = aa_hoards.hoard_id
        WHERE 
        CAST(b_start_date as decimal) >= {sd} and CAST(b_end_date as decimal) <= {ed}
        AND
        aa_hoards.hoard_id NOT IN (SELECT cross_reference FROM aa_parent_child)
        AND
        aa_hoards.hoard_id IN (SELECT aa_mints.hoard from aa_mints WHERE aa_mints.mint_uri = '{mintUri}')
        AND
        mint_uri != ''
        GROUP BY aa_hoards.hoard_id, mint_uri
        ORDER BY mint_uri; 
    """.format(sd = sd, ed = ed, mintUri=mintUri)    
    
    dfHoardEgo = pd.read_sql_query(sql, cnx)
    return dfHoardEgo

#this displays a graph of all of the coin hoards connected with a mint
def displayCoinEgoHoardGraph (coinHoardGraph, coinHoardGraphDict, maxNodeSize, MinNodeSize, maxEdgeSize, MinEdgeSize, figXsize, figYSize, nodes_df, FontSize):
    
    oldmax = int(max(coinHoardGraphDict.values()))
    oldmin = int(min(coinHoardGraphDict.values()))
    
    for k, v in coinHoardGraphDict.items():
        coinHoardGraphDict[k] = linReScale(oldmax, oldmin, maxNodeSize, MinNodeSize, v)
    
    edgeweights = [i['count'] for i in dict(coinHoardGraph.edges).values()]
    oldmax = max(edgeweights)
    oldmin = min(edgeweights)
    
    edgeweights = [linReScale(oldmax, oldmin, maxEdgeSize, MinEdgeSize, a) for a in edgeweights]
    
    labelcopy = pd.DataFrame()
    labelcopy = nodes_df.copy()
    labellist = labelcopy.filter(['mint_uri','title'])
    labellistDict = pd.Series(labellist.title.values,index=labellist.mint_uri).to_dict()

    
    fig, ax = plt.subplots(figsize=(figXsize,figYSize))
    pos = nx.nx_agraph.graphviz_layout(coinHoardGraph, prog='neato')
    
    nx.draw_networkx_nodes(coinHoardGraph, pos, ax = ax, edgecolors = 'black', node_color=colors, cmap=my_colors, labels=True, node_size=[v * 50 for v in coinHoardGraphDict.values()])
    nx.draw_networkx_edges(coinHoardGraph, pos, width=edgeweights, ax=ax)
    nx.draw_networkx_labels(coinHoardGraph, pos, labellistDict, font_size=FontSize,font_color='black')
    
    plt.show()

#this creates dataframes from our graph data to use for maps and other 
def createMapDfs(dfEgoHoard, partition, graph, colormap):
    smallhoarddf = dfEgoHoard.groupby( ["location_uri", "hoard_id"] ).count().reset_index()
    sql = """
    SELECT aa_locations.id AS location_uri, aa_locations.title, aa_locations.lat, aa_locations.lon FROM aa_locations;
    """
    
    locdf = pd.DataFrame()
    locdf = pd.read_sql_query(sql, cnx)
    smallhoarddf = smallhoarddf.merge(locdf, how='left')
    del smallhoarddf['hoard_title']
    del smallhoarddf['mint_title']
    del smallhoarddf['mint_uri']
    del smallhoarddf['count']
    smallhoarddf.rename(columns={'location_uri':'mint_uri'}, inplace=True)
    smallhoarddf['title'] = smallhoarddf['hoard_id']
    mapDf = pd.DataFrame(coinHoardGraph.nodes, columns =['mint_uri']) 
    partdf = pd.DataFrame(list(partition.items()),columns = ['mint_uri','partition'])
    partdf['color'] = partdf.apply (lambda row: makeNetColor(row, my_colors), axis=1)
    mapDf = mapDf.merge(partdf, how='left')
    weightdf = pd.DataFrame(list(coinHoardGraph.degree(weight='weight')),columns = ['mint_uri','degree']) 
    mapDf = mapDf.merge(weightdf, how='left')
    
    sql = """
    SELECT aa_locations.id AS mint_uri, aa_locations.title, aa_locations.lat, aa_locations.lon 
    FROM aa_locations;
    """
    
    locdf = pd.DataFrame()
    locdf = pd.read_sql_query(sql, cnx)
    
    mapDf = mapDf.merge(locdf, how='left')
    partdf.rename(columns={'mint_uri':'hoard_id'}, inplace=True)
    weightdf.rename(columns={'mint_uri':'hoard_id'}, inplace=True)
    
    smallhoarddf = smallhoarddf.merge(partdf, how='left')
    smallhoarddf = smallhoarddf.merge(weightdf, how='left')
    
    del smallhoarddf['mint_uri']
    smallhoarddf.rename(columns={'hoard_id':'mint_uri'}, inplace=True)
    
    mapDf = mapDf[mapDf['lat'].notna()]
    
    smallhoarddf['lat'] = pd.to_numeric(smallhoarddf['lat'])
    smallhoarddf['lon'] = pd.to_numeric(smallhoarddf['lon'])
    smallhoarddf['degree'] = pd.to_numeric(smallhoarddf['degree'])

    mapDf['lat'] = pd.to_numeric(mapDf['lat'])
    mapDf['lon'] = pd.to_numeric(mapDf['lon'])
    mapDf['degree'] = pd.to_numeric(mapDf['degree'])

    
    dataframeResults['mintsDf'] = mapDf
    dataframeResults['hoardsDf'] = smallhoarddf
    
    mapDf = mapDf.merge(smallhoarddf, how='outer')
    mapDf = mapDf[(mapDf['lat']!="lat")]
    mapDf = mapDf[(mapDf['lat']!="")]
    
    dataframeResults['mapDf'] = mapDf
    
    return dataframeResults

#this function populates a feature group from a datatable, rescaleas it, then adds it to the map

def addFeatureGrouptoMap(df, featuregroup, nodeMaxSize, nodeMinSize, nodeAttribute, nodeTitle, m):
    
    oldmax = df[nodeAttribute].max()
    oldmin = df[nodeAttribute].min()

    for index, row in df.iterrows():
        if float(row['lat']) < 9999999:
            radius = linReScale(oldmax, oldmin, nodeMaxSize, nodeMinSize, row['degree'])
            popupText = 'Title: {title}'.format(title = row[nodeTitle])
            featuregroup.add_child(folium.CircleMarker(location = [row['lat'], row['lon']],
                                popup = popupText,
                                radius = radius,
                                fill_color=row['color'], 
                                color = 'black', 
                                fill_opacity=0.7,
                                weight = .5
                               ))
            m.add_child(featuregroup) 

def addFeatureLabelstoMap(df, nodeTitle, m, fontsize):
    for index, row in df.iterrows():
        if float(row['lat']) < 9999999:
            folium.Marker(location = [row['lat'], row['lon']],
                          icon=DivIcon(
                              icon_size=(15,15),
                              icon_anchor=(0,0),
                              html='<div style="font-size: {fontsize}pt">{label}</div>'.format(label = row[nodeTitle], fontsize = fontsize))
                         ).add_to(m)

#this function maps all of the hoards between two given years, partitions them, and returns a dataframe           
def hoardsGroupMapper(startdate, enddate, buffer, mapcolorramp, featuregroup):
    sql = coinNetworkSql(startdate, enddate, buffer)
    dfmaptest = pd.DataFrame()
    dfmaptest = pd.read_sql_query(sql, cnx)
    mapgraph = nx.from_pandas_edgelist(dfmaptest, 'source', 'target', 'weight')
    partition = community.best_partition(mapgraph)
    modularity = community.modularity(partition, mapgraph)
    dfmap = createMapDataframe(dfmaptest, mapgraph, partition, mapcolorramp)
    return dfmap

### This function creates the dataframe necessary for the map out of all of the various measurements, etc.
def createMapDataframe(dataframe, graph, partition, color_map):
    sql = """
        SELECT aa_locations.id AS source, 
        aa_locations.title, 
        aa_locations.lat, 
        aa_locations.lon, 
        aa_locations.geom FROM aa_locations;
        """
    dataframe = pd.DataFrame()
    dataframe = pd.read_sql_query(sql, cnx)
    partdf = pd.DataFrame(list(partition.items()),columns = ['source','partition'])
    dataframe = dataframe.merge(partdf, how='left')
    weightdf = pd.DataFrame(list(graph.degree(weight='weight')),columns = ['source','weighted_degree']) 
    dataframe = dataframe.merge(weightdf, how='left')
    degreedf = pd.DataFrame(list(graph.degree()),columns = ['source','degree']) 
    dataframe = dataframe.merge(degreedf, how='left')
    dataframe = dataframe[dataframe.partition.notnull()]
    columncolor = dataframe[['source', 'partition']] 
    
    minima = min(columncolor['partition'].tolist())
    maxima = max(columncolor['partition'].tolist())
    norm = matplotlib.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap=color_map)
    
    aDict = {}
    #since for some reasin the color mapper is not callable within an iterated dataframe, we are going to do this the hard way
    for index, row in dataframe.iterrows():
        aDict[row['source']] = str(matplotlib.colors.to_hex(mapper.to_rgba(row['partition'])))
    
    colordf = pd.DataFrame(list(aDict.items()), columns=['source', 'color'])
    dataframe = dataframe.merge(colordf, how='left')
    
    sql = """
    SELECT aa_locations.id AS location_uri, aa_locations.lat, aa_locations.lon FROM aa_locations;
    """
    locdf = pd.DataFrame()
    locdf = pd.read_sql_query(sql, cnx)


    dataframe = dataframe.merge(locdf, how='left')
    dataframe[["lat", "lon"]] = dataframe[["lat", "lon"]].apply(pd.to_numeric)
    
    return dataframe

#this function grabs dynastic mint information from the ANS
def dynastymintlocator(year_start, year_end, dynasty):
    sql = """
        SELECT aa_mantis.mint, nomisma_mints.mint, nomisma_mints.pleiades, nomisma_mints.lat, nomisma_mints.long  
        FROM aa_mantis
        FULL OUTER JOIN nomisma_mints
        ON
        LOWER(aa_mantis.mint) = LOWER (nomisma_mints.label)
        WHERE
        aa_mantis.start_year BETWEEN {year_start} AND {year_end}
        AND
        aa_mantis.end_year BETWEEN {year_start} AND {year_end}
        AND
        LOWER(aa_mantis.dynasty) LIKE LOWER('%%{dynasty}%%')
        GROUP BY
        aa_mantis.mint,nomisma_mints.mint, nomisma_mints.pleiades, nomisma_mints.lat, nomisma_mints.long
        ;
        """.format(year_start = year_start,year_end = year_end,dynasty = dynasty)
    dataframe = pd.DataFrame()
    dataframe = pd.read_sql_query(sql, cnx)
    geodf = geopandas.GeoDataFrame(dataframe, geometry=geopandas.points_from_xy(dataframe.long, dataframe.lat))
    return geodf;

#I want to map the dynasties slightly differently, so they get their own folium group
def dynastyMintMapper(geodf, dyncolor, radius, featuregroup, m):
    for index, row in geodf[geodf.lat.notnull()].iterrows():
        featuregroup.add_child(
            folium.CircleMarker(
                location=[row['geometry'].xy[1][0], row['geometry'].xy[0][0]],
                fill=True,
                fill_color = dyncolor,
                color = dyncolor,
                opacity = 0.4,
                fill_opacity=0.4,
                radius = radius
            ))
    m.add_child(featuregroup)

        

    
#A funvtion tying together the pgRouting 
def polityRouteMaker (df, df_id, source_table, ego_table, ego):
    nodeList = []
    # now we need to get the actual vector value  on our nodded route network for each of the recognized place entries in the table
    for index, row in df.iterrows():
        sql = pgrPointFinder(row[df_id], source_table)
        results_set = cnx.execute(sql)
        # ugly hardcoded for now; may need to make this more flexible in the future
        # after the result is in, we add it to the list
        for id in results_set:
            nodeList.append(int(id[0]))
    placerecord = cnx.execute(pgrPointFinder (ego, ego_table))
    # some more ugly hard coding here; this should alwyas be the first result. I think/hope/etc...
    id_count = placerecord.first()[0]
    # now to get the pleiades ID and router match up
    sql = politicalPgrMaker(id_count, nodeList)
    # now to do the actual routing
    df = gpd.GeoDataFrame.from_postgis(sql, cnx, geom_col='geom' )
    return df

#finds the closest point in our routes network to a given goegraphic point
def pgrPointFinder (placeId, table):
    sql = """
        SELECT closest_pt.id
        FROM {table} a
        CROSS JOIN LATERAL
        (SELECT
        id,
        a.geom <-> b.the_geom AS dist
        FROM routes_single_noded_vertices_pgr b
        WHERE a.id = '{placeId}'
        ORDER BY a.geom <-> b.the_geom
        LIMIT 1) AS closest_pt;
        """.format(placeId = placeId, table=table)
    return sql


#actually makes the routes from nodes and edges
def politicalPgrMaker (target, connected_list):
    sql = """
    SELECT
    max(seq),
    edge,
    count(edge),
    geom
    FROM
        pgr_dijkstra(
        'SELECT id,
        source,
        target,
        st_length(geom) AS cost
        FROM routes_single_noded',
        array{connected_list},
        {target},
        false) AS pt
    JOIN routes_single_noded rd 
    ON pt.edge = rd.id
    GROUP BY
        edge,
        geom;
    """.format(target = target, connected_list = connected_list)
    return sql

#this function returns routes for a selected polity 
def ancientEmpireRoutes(polity, tableName, start_date, end_date):
    #stripping out any non-pleaides ids, as by definition these will not show up
    sql = """
    select places.id
    from places JOIN {tableName}
    ON places.id::varchar = {tableName}.source
    WHERE target ='{polity}'
    AND
    places.reprlatlong IS NOT NULL
    AND
    ({tableName}.start_date::double precision <= {end_date})
    AND
    ({tableName}.end_date::double precision >= {start_date})
    AND
    {tableName}.type IN ('capitol', 'hegemon of', 'control by', 'founded by', 'garrisoned by', 'subordinate')
    GROUP BY places.id
    """.format(polity = polity, start_date=start_date, end_date = end_date, tableName = tableName)
    df = pd.DataFrame()
    df = pd.read_sql_query(sql, cnx)
    return df

#this function returns places for a selected polity 
def ancientEmpirePoints(polity, tableName, start_date, end_date):
    #stripping out any non-pleaides ids, as by definition these will not show up
    sql = """
    select places.id, places.title, places.geom
    from places JOIN {tableName}
    ON places.id::varchar = {tableName}.source
    WHERE target ='{polity}'
    AND
    places.reprlatlong IS NOT NULL
    AND
    ({tableName}.start_date::double precision <= {end_date})
    AND
    ({tableName}.end_date::double precision >= {start_date})
    AND
    {tableName}.type IN ('capitol', 'hegemon of', 'control by', 'founded by', 'garrisoned by', 'subordinate')
    GROUP BY places.id
    """.format(polity = polity, start_date=start_date, end_date = end_date, tableName = tableName)
    df = pd.DataFrame()
    df = gpd.GeoDataFrame.from_postgis(sql, cnx, geom_col='geom' )
    return df

def obversetoreversecounter(mint):
    sql = """
    Select obvdie AS obverse_die, count(distinct revdie) as number_of_reverse_dies
    from
    all_coins
    where mint = '{mint}'
    AND
    obvdie IS NOT NULL
    AND
    obvdie != ''
    AND
    obvdie !='dupe'
    AND
    obvdie !='no_image'
    group by obvdie
    """.format(mint = mint)
    
    df = pd.DataFrame()
    df = pd.read_sql_query(sql, cnx)
    
    #we usually have mixed numbers and characters for naming conventions; this should arrange everythign nicely
    df.index = df['obverse_die'].str.replace('[a-zA-Z_]+','').astype(float)
    df = df.sort_index().reset_index(drop=True)
    return df


def obversetoreversecounter(mint):
    sql = """
    Select obvdie AS obverse_die, count(distinct revdie) as number_of_reverse_dies
    from
    all_coins
    where mint = '{mint}'
    AND
    obvdie IS NOT NULL
    AND
    obvdie != ''
    AND
    obvdie !='dupe'
    AND
    obvdie !='no_image'
    group by obvdie
    """.format(mint = mint)
    
    df = pd.DataFrame()
    df = pd.read_sql_query(sql, cnx)
    
    #we usually have mixed numbers and characters for naming conventions; this should arrange everythign nicely
    df.index = df['obverse_die'].str.replace('[a-zA-Z_]+','').astype(float)
    df = df.sort_index().reset_index(drop=True)
    return df


def obverseweightaverager(mint):
    sql = """
    Select obvdie AS obverse_die, AVG(CAST(weight as decimal)) as average_weight
    from
    all_coins
    where mint = '{mint}'
    AND
    obvdie IS NOT NULL
    AND
    obvdie != ''
    AND
    obvdie !='dupe'
    AND
    obvdie !='no_image'
    group by obvdie
    """.format(mint = mint)
    
    df = pd.DataFrame()
    df = pd.read_sql_query(sql, cnx)
    
    #we usually have mixed numbers and characters for naming conventions; this should arrange everythign nicely
    df.index = df['obverse_die'].str.replace('[a-zA-Z_]+','').astype(float)
    df = df.sort_index().reset_index(drop=True)
    return df

def obversecoincounter(mint):
    sql = """
    Select obvdie AS obverse_die, count(obvdie) AS number_of_coins
    from
    all_coins
    where mint = '{mint}'
    AND
    obvdie IS NOT NULL
    AND
    obvdie != ''
    AND
    obvdie !='dupe'
    AND
    obvdie !='no_image'
    group by obvdie
    """.format(mint = mint)
    
    df = pd.DataFrame()
    df = pd.read_sql_query(sql, cnx)
    
    #we usually have mixed numbers and characters for naming conventions; this should arrange everythign nicely
    df.index = df['obverse_die'].str.replace('[a-zA-Z_]+','').astype(float)
    df = df.sort_index().reset_index(drop=True)
    return df


def obverseToTypeList (mint):
    sql = """
    Select obvdie AS obverse_die, group_concat(DISTINCT typeid) as price_types
    from
    all_coins
    where mint = '{mint}'
    AND
    obvdie IS NOT NULL
    AND
    obvdie != ''
    AND
    obvdie !='dupe'
    AND
    obvdie !='no_image'
    AND
    typeid !=''
    AND
    typeid IS NOT NULL
    group by obvdie
    """.format(mint = mint)
    
    df = pd.DataFrame()
    df = pd.read_sql_query(sql, cnx)
    df = sortMixedDatatable(df)

    return df

def obvinfochart(mint):
    dfObverseReverseCount = pd.DataFrame()
    dfObverseReverseCount = obversetoreversecounter(mint)
    dfobvprice = pd.DataFrame()
    dfobvprice = obverseToTypeList(mint)
    dfobvavgweight = pd.DataFrame()
    dfobvavgweight = obverseweightaverager(mint)
    dfobvcoincount = pd.DataFrame()
    dfobvcoincount = obversecoincounter(mint)
    dfObvinfo = pd.DataFrame()
    dfObvinfo = dfobvcoincount.copy()
    dfObvinfo = dfObvinfo.merge(dfObverseReverseCount, how='left')
    dfObvinfo = dfObvinfo.merge(dfobvavgweight, how='left')
    dfObvinfo = dfObvinfo.merge(dfobvprice, how='left')
    dfObvinfo.set_index('obverse_die', inplace=True)
    return dfObvinfo

def sortMixedDatatable (df):
    #we usually have mixed numbers and characters for naming conventions; this should arrange everythign nicely
    df.index = df['obverse_die'].str.replace('[a-zA-Z_]+','').astype(float)
    df = df.sort_index().reset_index(drop=True)
    return df


#this function creates the big dataframe for all of the routes
def createBigNetworkDf():
    sql = """
    SELECT aa_mints.mint_uri as source, a.location_uri as target, aa_mints.mint as label, 
    aa_mints.count as weight, 
    CONCAT(LPAD((@a.b_start_date)::text, 4, '0'), ' ',start_stamp ) AS start_date,
    CONCAT(LPAD((@a.b_end_date)::text, 4, '0'), ' ',end_stamp ) AS end_date
    FROM aa_mints JOIN
    (SELECT aa_hoards.hoard_id, aa_hoards.location_uri, aa_locations.geom, b_start_date,
    CASE
    WHEN  b_start_date::varchar LIKE '-%%' THEN 'BC'
    ELSE 'AD'
    END 
    AS start_stamp, 
    b_end_date,
    CASE
    WHEN  b_end_date::varchar LIKE '-%%' THEN 'BC'
    ELSE 'AD'
    END 
    AS end_stamp
    FROM aa_hoards LEFT JOIN aa_locations 
    ON LOWER(aa_hoards.location_uri) = LOWER (aa_locations.id) 
    WHERE aa_hoards.hoard_id NOT IN (SELECT cross_reference from aa_parent_child)
    AND b_start_date IS NOT NULL
    ) a
    
    ON aa_mints.hoard = a.hoard_id
    WHERE
    aa_mints.mint_uri IS NOT NULL 
    AND aa_mints.mint_uri <> '' 
    AND a.location_uri IS NOT NULL 
    AND a.location_uri <> ''
    """
    
    df = pd.DataFrame()
    df = pd.read_sql_query(sql, cnx)
    df["weight"] = pd.to_numeric(df["weight"])
    df["weight"].fillna(1)
    df["weight"] = df["weight"].replace(np.nan, 1, regex=True)
    df["weight"].replace(r'^\s*$', 1, regex=True)
    return df

#this function inserts the dataframe into the database in the format required for QGIS
def loadBigRouteFrame(df):
    for index, row in df.iterrows():
        sql = """
        SELECT '{mint}' as mint, '{hoard}' as hoard, '{weight}' as weight, '{start_date}' as start_date, '{end_date}' as end_date, ST_AsText(st_union (a.geom)) as geom FROM
        (SELECT max(seq), edge, count(edge), geom
        FROM pgr_dijkstra(
        'SELECT id, source, target, st_length(geom) as cost FROM routes_single_noded',
    (select array_agg(closest_pt.id)
    from (SELECT q.mint_uri AS source, q.location_uri AS target, r.geom 
    from (select aa_mints.mint_uri, aa_hoards.hoard_id, aa_hoards.location_uri 
    FROM aa_mints JOIN aa_hoards 
    ON aa_mints.hoard = aa_hoards.hoard_id 
    WHERE aa_hoards.hoard_id NOT IN 
    (SELECT cross_reference from aa_parent_child) 
    AND 
    aa_mints.mint_uri <> '' 
    AND 
    aa_mints.mint_uri IS NOT NULL 
    AND 
    aa_hoards.location_uri <> '' 
    AND 
    aa_hoards.location_uri IS NOT NULL
    ) q
    JOIN
    (SELECT aa_locations.geom, aa_locations.id from aa_locations)r 
    ON 
    q.location_uri = r.id 
    WHERE r.geom IS NOT NULL) a 
    CROSS JOIN LATERAL
    (SELECT
    id , 
    a.geom <-> b.the_geom as dist
    FROM routes_single_noded_vertices_pgr b
    WHERE a.source = '{mint}'
    ORDER BY a.geom <-> b.the_geom 
    LIMIT 1) AS closest_pt), 
    (select closest_pt.id 
    from aa_locations a 
    CROSS JOIN LATERAL 
    (SELECT 
    id, 
    a.geom <-> b.the_geom as dist 
    FROM routes_single_noded_vertices_pgr b 
    WHERE a.id = '{mint}' 
    ORDER BY a.geom <-> b.the_geom 
    LIMIT 1) AS closest_pt
    ), false
    ) as pt 
    JOIN routes_single_noded rd ON pt.edge = rd.id 
    GROUP BY edge, geom) a; 
    """.format(mint = row['source'], hoard = row['target'], weight = row['weight'], start_date =row['start_date'] , end_date = row['end_date'] )
    df2 = pd.DataFrame()
    df2 = pd.read_sql(sql, cnx)
    sqlInsert = """
        INSERT INTO aa_coin_routes(mint, hoard, weight, start_date, end_date, geom)
        VALUES('{mint}', '{hoard}', {weight}, '{start_date}', '{end_date}', st_multi(ST_GeomFromText('{geom}', 4326)));
        """.format(mint = df2['mint'].values[0], hoard = df2['hoard'].values[0], weight = df2['weight'].values[0], start_date = df2['start_date'].values[0], end_date = df2['end_date'].values[0], geom = df2['geom'].values[0])
    try:
        cnx.execute(sqlInsert)
    except:
        print("error: {}".format(sqlInsert))

def typestoObverseCounter(mint):
    sql = """
    Select typeid AS type, count(distinct obvdie) as number_of_obverse_dies
    from
    all_coins
    where mint = '{mint}'
    AND
    obvdie IS NOT NULL
    AND
    obvdie != ''
    AND
    obvdie !='dupe'
    AND
    obvdie !='no_image'
    AND
    typeid IS NOT NULL
    AND
    typeid != ''
    group by type
    """.format(mint = mint)
    
    df = pd.DataFrame()
    df = pd.read_sql_query(sql, cnx)
    
    #we usually have mixed numbers and characters for naming conventions; this should arrange everythign nicely
    df.index = df['type']
    df = df.sort_index().reset_index(drop=True)
    return df


def typescoincounts(mint):
    sql = """
    Select typeid AS type, count(distinct id) as number_of_coins
    from
    all_coins
    where mint = '{mint}'
    AND
    obvdie IS NOT NULL
    AND
    obvdie != ''
    AND
    obvdie !='dupe'
    AND
    obvdie !='no_image'
    AND
    typeid IS NOT NULL
    AND
    typeid != ''
    group by typeid
    """.format(mint = mint)
    
    df = pd.DataFrame()
    df = pd.read_sql_query(sql, cnx)
    
    #we usually have mixed numbers and characters for naming conventions; this should arrange everythign nicely
    df.index = df['type']
    df = df.sort_index().reset_index(drop=True)
    return df


def typesweightaverager(mint):
    sql = """
    Select typeid AS type, AVG(CAST(weight as decimal)) as average_weight
    from
    all_coins
    where mint = '{mint}'
    AND
    obvdie IS NOT NULL
    AND
    obvdie != ''
    AND
    obvdie !='dupe'
    AND
    obvdie !='no_image'
    AND
    typeid IS NOT NULL
    AND
    typeid != ''
    group by typeid
    """.format(mint = mint)
    
    df = pd.DataFrame()
    df = pd.read_sql_query(sql, cnx)
    
    #we usually have mixed numbers and characters for naming conventions; this should arrange everythign nicely
    df.index = df['type']
    df = df.sort_index().reset_index(drop=True)
    return df


def typeinfochart(mint):
    dfObverseCount = pd.DataFrame()
    dfObverseCount = typestoObverseCounter(mint)
    dfCoinCount = pd.DataFrame()
    dfCoinCount = typescoincounts(mint)
    dfavgweight = pd.DataFrame()
    dfavgweight = typesweightaverager(mint)
    dfObvinfo = pd.DataFrame()
    dfObvinfo = dfCoinCount.copy()
    dfObvinfo = dfObvinfo.merge(dfObverseCount, how='left')
    dfObvinfo = dfObvinfo.merge(dfavgweight, how='left')
    dfObvinfo.set_index('type', inplace=True)
    return dfObvinfo


## Pure SQL
Functions

In [ ]:
# SQL to get the mints / hoards as source, target with counts for weights
def coinNetworkSql(startdate, enddate, buffer):
    sd = startdate - buffer;
    ed = enddate - buffer;
    sql = """
        SELECT  aa_mints.mint_uri as source, 
        aa_hoards.location_uri as target, 
        LOWER(aa_mints.mint) as label, 
        SUM(aa_mints.count) as weight
    FROM
    aa_mints
    LEFT JOIN
    aa_hoards
    ON
    aa_mints.hoard = aa_hoards.hoard_id
    WHERE 
    CAST(b_start_date as decimal) >= {sd} and CAST(b_end_date as decimal) <= {ed}
    AND
    aa_hoards.hoard_id NOT IN (SELECT cross_reference FROM aa_parent_child)
    AND
    aa_mints.mint_uri IS NOT NULL 
    AND 
    aa_mints.mint_uri <> ''
    AND aa_hoards.location_uri <> ''
    GROUP BY source, target, label
    ORDER BY label; 
    """.format(sd = sd, ed = ed)
    return sql


# Traditional Catalog

In the notebook this only displays a selection; this can be exported to a csv to be styled and sorted as desired

In [ ]:
coinCatdf = pd.DataFrame()
coinCatdf = coinCatalogWorking(mint["myrina"])
display(coinCatdf)



## Export as a .csv

In [ ]:
coinCatdf.to_csv('myrina_catalog.csv')

## List a Specific Type

In [ ]:
coinType = '1659'

coinCatdf.loc[coinCatdf['type'] == coinType]

## Types and Weights as a table

In [ ]:
means = coinCatdf.groupby(['type'])['weight'].mean()

means

## Average Weight Per Type

In [ ]:
ax = means.plot(kind = 'bar')
ax.set_ylabel('Mean Weight')

## Histogram for all weights

In [ ]:
coinCatdf.hist(column='weight')


## Histogram for weights by type

In [ ]:
coinType = '1659'

coinCatdf.loc[coinCatdf['type'] == coinType].hist(column='weight')


## Count number of coins in a type

In [ ]:
ax = coinCatdf['type'].value_counts().plot(kind='bar')


## Count number of reverse dies per obverse die

In [ ]:
# As usual the logic is contained in a function above
dfObverseReverseCount = pd.DataFrame()
dfObverseReverseCount = obversetoreversecounter(mint["myrina"])
dfObverseReverseCount

In [ ]:
# As usual the logic is contained in a function above
dfobvprice = pd.DataFrame()

dfobvprice = obverseToTypeList(mint["myrina"])
dfobvprice



In [ ]:
#average weights by die
dfobvavgweight = pd.DataFrame()

dfobvavgweight = obverseweightaverager(mint["myrina"])
dfobvavgweight

In [ ]:
#average weights by die
dfobvcoincount = pd.DataFrame()

dfobvcoincount = obversecoincounter(mint["myrina"])
dfobvcoincount

Of course, we can put these all together in whatever format is desired for viewing / publication:

In [ ]:
dfObvinfo = pd.DataFrame()
dfObvinfo = obvinfochart(mint["myrina"])
dfObvinfo

And we can do the same for types

In [ ]:
dftypeinfo = pd.DataFrame()
dftypeinfo = typeinfochart(mint["myrina"])
dftypeinfo

In [ ]:
dfObvinfo = pd.DataFrame()
dfObvinfo = obvinfochart(mint["kyme"])
dfObvinfo

In [ ]:
dfObvinfo = pd.DataFrame()
dfObvinfo = obvinfochart(mint["temnos"])
dfObvinfo

# Coins as networks

There are several different networks that can be created from our coin data. We can connect coins to types, dies (obverse and reverse) to types, coins to dies, and obverse to reverse dies. Each network type, with some different means of display and evaluation, are given below.

## Obverse to types

In [ ]:
coinCatdf

In [ ]:
coinGraph = nx.from_pandas_edgelist(coinCatdf, source='obverse', target='type')
coinGraphDegree = dict(coinGraph.degree)


## Traditional bipartite graph with a twist: Circles are sized according to number of connections

In [ ]:
plt.figure(figsize=(20,20))


pos = nx.bipartite_layout(coinGraph, coinCatdf['obverse'].tolist())

nx.draw(coinGraph,pos, node_size=[v * 200 for v in coinGraphDegree.values()])

nx.draw_networkx_labels(coinGraph,pos, font_size=20)

plt.show()


## Die Linkages as a network

In [ ]:
plt.figure(figsize=(12,12))

pos = nx.nx_agraph.graphviz_layout(coinGraph, prog='neato')
nx.draw(coinGraph, pos, node_size=[v * 100 for v in coinGraphDegree.values()])
nx.draw_networkx_labels(coinGraph,pos,font_size=14)
plt.show()

## Obverse to Reverse Links

In [ ]:
coinGraph = nx.from_pandas_edgelist(coinCatdf, source='obverse', target='reverse')
coinGraphDegree = dict(coinGraph.degree)


In [ ]:
plt.figure(figsize=(20,20))


pos = nx.bipartite_layout(coinGraph, coinCatdf['obverse'].tolist())

nx.draw(coinGraph,pos, node_size=[v * 200 for v in coinGraphDegree.values()])

nx.draw_networkx_labels(coinGraph,pos, font_size=20)

plt.show()

## Coins to Obverse Dies


In [ ]:
coinGraph = nx.from_pandas_edgelist(coinCatdf, source='id', target='obverse')
coinGraphDegree = dict(coinGraph.degree)

In [ ]:
plt.figure(figsize=(25,25))


pos = nx.bipartite_layout(coinGraph, coinCatdf['obverse'].tolist())

# Pass that layout to nx.draw
nx.draw(coinGraph,pos, node_size=[v * 200 for v in coinGraphDegree.values()])

nx.draw_networkx_labels(coinGraph,pos, font_size=20)

plt.show()

# Creating a Website

We can use d3js and this data to create an application that allows users to sort the different coin networks, get information on the coins, and interact with the data which can be placed into any website that allows javascript. The following code will perform the required transformations.

## Creating a nodes list

First, we must create a nodes list that has the following columns as nodes:
1. Coins
1. Obverse dies
1. Reverse dies
1. Coin types

This will be a python dictionary that is later converted into a .json object. We will build this with SQL queries so it can be run live as the database changes.

In [ ]:
#The actual logic is contained in our function block; here we are showing how to export the results
finalJson = makeHtmlJson(mint["kyme"])

text_file = open("kyme.json", "w")
text_file.write(finalJson)
text_file.close()



The .json file can now be placed in the /data directory of the coin showcase application; for more information see https://github.com/Aeolian-Alexanders/website

# Hoards

We can use these same tools and approaches to study hoards. We will first begin with ways to query hoards and perform statistical analysis; we will then move on to seeing how individual mints fit into an "ego network" of related hoards; and finally we will view all hoard data as a geospatial network using tools like PgRouting. 

## Hoard Information

Totals are computed by the query; they are broken down by type and denomenation to enable more filtering or sorting if required. For die studies, it is advisable to filter based on types or denomenations. Although the current data is incomplete, it is my hope that inclusion in a future CoinHoards release by the ANS, along with making the data publicly accessable will spur further devlopment / interest in completing the records.

In [ ]:
start_date = -282
end_date = -129
buffer = 2

dfHoard = pd.DataFrame()
dfHoard = hoardsDisplay (mint["myrina"], start_date, end_date, buffer)
dfHoard

## Basic histogram of number of coins found in each hoard

In [ ]:
dfHoard.hist(column='count')

## Map all Hoards associated with a mint

The logic is contained within the function; this is showing how it is called.

In [ ]:
MinimumMarkerSize = 5
MaximumMarkerSize = 25

m = createmapvis()
mintHoardMapper(dfHoard, "count", (mint["myrina"]), MinimumMarkerSize, MaximumMarkerSize, m)
m

## Hoards as Networks

Now we are going to model hoards as networks; the mints and hoards will be nodes connecting to each other, with the number of coins of each mint as the weight. We will begin with an ego network showing just the mints that are associated with our mint of interest

### Create an ego network surrounding the mint of interest

In [ ]:
start_date = -282
end_date = -129
buffer = 2

dfEgoHoard = pd.DataFrame()
dfEgoHoard = hoardsEgoDisplay (mint["myrina"], start_date, end_date, buffer)

In [ ]:
dfEgoHoard

While this is nice for seeing the data in a tabular form, we will need to do some woek on the table and the graph before we display it. What we will do is get network statistics for the graph, THEN break it into different dataframes which are brought into a new graph for display. We want to set our colors based on the partition, and then keep those same colors for mapping the nodes.

First, we will create our ego network by splitting the data frame, and run all of the partition / modularity off of it

We want to get the weight of our edges, so the foloowing code pulls data form our main list and forms a new dataframe with the weight of the conecction derived from the total number of coins

In [ ]:
dfEgoHoardEdge = dfEgoHoard.groupby(['mint_uri', 'hoard_id',]).sum().reset_index()

In [ ]:
dfEgoHoardEdge

This may seem counter-intuitive, but we can use a "staging" graph to break apart our main table into nodes, which we then build another dataframe around. We are doing it this way to pull less data back and forth from the database. We are going to peform som network analysis on this, then pull apart the nodes with properties that can be used in folium and other libraries without having to worry about changing colors, etc

In [ ]:
coinHoardGraph = nx.from_pandas_edgelist(dfEgoHoardEdge, 'mint_uri', 'hoard_id','count')
coinHoardGraphDict = dict(coinHoardGraph.degree)
partition = community.best_partition(coinHoardGraph)
modularity = community.modularity(partition, coinHoardGraph)

colors = [partition[n] for n in coinHoardGraph.nodes()]
my_colors = plt.cm.Set2 # you can select other color pallettes here: https://matplotlib.org/users/colormaps.html



Now, rebuild the nodes and edges tables from the network. We will use these to build our maps, etc. We are also going to have to split the hoard and the mints to make this work

In [ ]:
dataframeResults = {}
dataframeResults = createMapDfs(dfEgoHoard, partition, coinHoardGraph, my_colors)


The hoards dataframe contains information about hoards

In [ ]:
dataframeResults['hoardsDf']

The mints dataframe contains information about mints

In [ ]:
dataframeResults['mintsDf']

The map dataframe ties together everything

In [ ]:
dataframeResults['mapDf']

In [ ]:
displayCoinEgoHoardGraph (coinHoardGraph, coinHoardGraphDict, 30, 5, 4, .5, 20, 20, dataframeResults['mapDf'], 12)

Using our function, we can map all hoards and mints:

In [ ]:
m = createmapvis()

#we are going to copy the dataframe so we can change the color

hoardsinputdf = dataframeResults['mapDf'].copy()
hoardsinputdf['color'] = hoardsinputdf.apply (lambda row: makeNetColor(row, my_colors), axis=1)

hoardsdf = pd.DataFrame()
hoardsfg = folium.FeatureGroup(name='Coin Hoards Network: {mint}'.format(mint = 'myrina' ))
addFeatureGrouptoMap(hoardsinputdf, hoardsfg, 30, 5, 'degree', 'title', m)
#optional labels - not really necessary when the map is interactive
addFeatureLabelstoMap(hoardsinputdf, 'title', m, 5)
m.add_child(folium.LayerControl())
m

Or just the mints / hoards as desired:

In [ ]:
m = createmapvis()

#we are going to copy the dataframe so we can change the color

hoardsinputdf = dataframeResults['mintsDf'].copy()
hoardsinputdf['color'] = hoardsinputdf.apply (lambda row: makeNetColor(row, my_colors), axis=1)

hoardsdf = pd.DataFrame()
hoardsfg = folium.FeatureGroup(name='Coin Hoards Network: {mint}'.format(mint = 'myrina' ))
addFeatureGrouptoMap(hoardsinputdf, hoardsfg, 30, 5, 'degree', 'title', m)
m

## All hoards at a given time in a network

We can use the tools and data we created to see all mints that are represented in hoards between any arbitrary diates. The idea here is to see how the partitions (communities) changed over time.

We can also map mints that ANS MANTIS identiifed as belonging to a certain dynasty

In [ ]:
startdate = third_syrian_war['start']
enddate = third_syrian_war['end']
buffer = 5

m = createmapvis()

hoardsdf = pd.DataFrame()
hoardsfg = folium.FeatureGroup(name='Coin Hoards Network: {startdate} to {enddate}'.format(startdate = startdate - buffer,enddate = enddate + buffer ))
hoardsdf = hoardsGroupMapper(startdate, enddate, buffer, my_colors, hoardsfg)
addFeatureGrouptoMap(hoardsdf, hoardsfg, 30, 5, 'degree', 'title', m)

ptolemyfg = folium.FeatureGroup(name='Ptolemaic Mints')
ptolemydf = geopandas.GeoDataFrame()
ptolemydf = dynastymintlocator(startdate, enddate, 'ptolem')
dynastyMintMapper(ptolemydf,'orange', 5, ptolemyfg, m)


seleucidfg = folium.FeatureGroup(name='Seleucid Mints')
seleuciddf = geopandas.GeoDataFrame()
seleuciddf = dynastymintlocator(startdate, enddate, 'seleuc')
dynastyMintMapper(ptolemydf,'black', 5, seleucidfg, m)

m.add_child(folium.LayerControl())
m



# Creating Abstract Route Networks

# NOTE: These will ONLY work in a postgre/postgis databse! They are here for reference, and to serve as a model for further exploration

We can abstract the flow of coins to hoards by using the pgRouting library, along with different LOD resources (in our case routes from ORBIS, roads from AWMC/DARMC, and roads from Harvard World Map). The idea here is to create a visual "flow" diagram that illustrates quantities of coins moving through economic networks. As the coins almost certainly did not go straight from the mint to a hoard, this is a visual abstrction that attempts to see potentially important routes

This first function makes a route network from a given ego to hoards

In [ ]:
dfHoard

In [ ]:
#gdf = geopandas.GeoDataFrame()
gdf = polityRouteMaker (dfHoard, 'location_uri', 'aa_locations', 'places', mint["kyme"])
gdf

This should be mapped; a basic exmaple is given below:

In [ ]:
m = createmapvis()

folium.Choropleth(
    gdf,
    line_weight=3,
    line_color='blue'
).add_to(m)

m

These routes can be added to our exsisting maps:

In [ ]:
m = createmapvis()

#we are going to copy the dataframe so we can change the color

hoardsinputdf = dataframeResults['hoardsDf'].copy()
hoardsinputdf['color'] = hoardsinputdf.apply (lambda row: makeNetColor(row, my_colors), axis=1)

hoardsdf = pd.DataFrame()
hoardsfg = folium.FeatureGroup(name='Coin Hoards Network: {mint}'.format(mint = 'myrina' ))
addFeatureGrouptoMap(hoardsinputdf, hoardsfg, 30, 5, 'degree', 'title', m)

folium.Choropleth(
    gdf,
    line_weight=3,
    line_color='blue'
).add_to(m)

m

To continue studying routes as networks, we can build this for *all* mints and hoards in the database. This is far too complex to display in a jupyter/folium combination, but the data can be brought into QGIS or other GIS desktop software. To facilitate this, we are also changing the dates so they can be read by the QGIS timeseries plugin.

The functions for doing so are shown below; these will modify the data on your machine. The intent is to show the process and data structures, so they can be applied to different investigations.

We will show the big dataaframe here to get an idea of all of the mints to hoards connections that we can map. This will grow as more mints are assigned places.

In [ ]:
df = pd.DataFrame()
df = createBigNetworkDf()
df

# Beyond Coins: Mapping Ancient Polities

The routes idea from the hoards can be scaled up to look at probable routes and axes of communication for larger political entities. In this case, we are taking gazetteers of connections (political, social, etc) uncovered by this investigation, plotting the known places mentioned in the gazetteers, and then constructing route lines like we did between mints and hoards. This allows us to visualize and consider the political (and social) networks that were also at play at the same time.

For our example, we will look at Pergamon during its earliest times, then its larger extent after the peace of Apamea in 188 BCE.

In [ ]:
polity = 550812

df = ancientEmpireRoutes(polity, 'aa_hellenistic_edges', -262, -197)
dfpoints = ancientEmpirePoints(polity, 'aa_hellenistic_edges', -262, -197)

dfRoutes = gpd.GeoDataFrame = polityRouteMaker (df, 'id', 'places', 'places', polity )


In [ ]:
m = createmapvis()

folium.Choropleth(
    dfRoutes,
    line_weight=3,
    line_color='red'
).add_to(m)

m

In [ ]:
polity = 550812

df = ancientEmpireRoutes(polity, 'aa_hellenistic_edges', -190, -128)
dfpoints = ancientEmpirePoints(polity, 'aa_hellenistic_edges', -190, -128)

dfRoutes = gpd.GeoDataFrame = polityRouteMaker (df, 'id', 'places', 'places', polity)


In [ ]:
m = createmapvis()

folium.Choropleth(
    dfRoutes,
    line_weight=3,
    line_color='red'
).add_to(m)

m

Due to limitations in folium, it is currently nearlly impossible to style the lines by the number of times they are crossed by routes; we can do this with another library but we can not use the accurate backround. As an alternative, we can bring these routes into QGIS by simply performing the same queries there.

In [ ]:
ax = dfRoutes.plot(figsize=(10, 10), alpha=0.5, edgecolor='k', linewidth=dfRoutes['count'])
ctx.add_basemap(ax, crs=dfRoutes.crs)

This higlights an important aspect of the view: potentially important areas are crossed by more routes, leading to larger lines. We can combine these to see a network of coins against a political entitiy:

In [ ]:
m = createmapvis()

folium.Choropleth(
    dfRoutes,
    line_weight=3,
    line_color='red'
).add_to(m)

folium.Choropleth(
    gdf,
    line_weight=3,
    line_color='blue'
).add_to(m)


m

In this case, the routes show that Myrina's coinage stretchedd far beyond the confines of the Attalid kingdom, and is far more oriented to the heartland of the Seleucid Empire, which previously dominated the area.


# Tying it all together

So, what about aeolian economic activity during some of the most intensive production of Temnos? We can see the silver flows from Temnos when compared with the axes of communication of the Attalids and the diminshed Seleukid Empire from 153 to 145 BCE. We only have limited data on when certian communites changed hands; this project was overly conservative when creating the networks, and only considered cases where imperial power was explicitly revealed in literature, epigraphy, or papyri. Once again this data can be steadily improved with more community involvement.

In [ ]:
start_date = -153
end_date = -145
buffer = 2

m = createmapvis()

dfEgoHoard = pd.DataFrame()
dfEgoHoard = hoardsEgoDisplay (mint["temnos"], start_date, end_date, buffer)
dfEgoHoardEdge = dfEgoHoard.groupby(['mint_uri', 'hoard_id',]).sum().reset_index()

coinHoardGraph = nx.from_pandas_edgelist(dfEgoHoardEdge, 'mint_uri', 'hoard_id','count')
coinHoardGraphDict = dict(coinHoardGraph.degree)
partition = community.best_partition(coinHoardGraph)
modularity = community.modularity(partition, coinHoardGraph)

colors = [partition[n] for n in coinHoardGraph.nodes()]
my_colors = plt.cm.Set2 # you can select other color pallettes here: https://matplotlib.org/users/colormaps.html

dataframeResults = {}
dataframeResults = createMapDfs(dfEgoHoard, partition, coinHoardGraph, my_colors)


hoardsinputdf = dataframeResults['hoardsDf'].copy()
hoardsinputdf['color'] = hoardsinputdf.apply (lambda row: makeNetColor(row, my_colors), axis=1)

hoardsdf = pd.DataFrame()
hoardsfg = folium.FeatureGroup(name='Coin Hoards Network: {mint}'.format(mint = 'temnos' ))
addFeatureGrouptoMap(hoardsinputdf, hoardsfg, 30, 5, 'degree', 'title', m)

gdfTemnos = polityRouteMaker (dfEgoHoard, 'location_uri', 'aa_locations', 'places', mint["temnos"])



In [ ]:

folium.Choropleth(
    gdfTemnos,
    line_weight=3,
    line_color='black'
).add_to(m)

In [ ]:
start_date = -153
end_date = -145
buffer = 2

m = createmapvis()

dfEgoHoard = pd.DataFrame()
dfEgoHoard = hoardsEgoDisplay (mint["myrina"], start_date, end_date, buffer)
dfEgoHoardEdge = dfEgoHoard.groupby(['mint_uri', 'hoard_id',]).sum().reset_index()

coinHoardGraph = nx.from_pandas_edgelist(dfEgoHoardEdge, 'mint_uri', 'hoard_id','count')
coinHoardGraphDict = dict(coinHoardGraph.degree)
partition = community.best_partition(coinHoardGraph)
modularity = community.modularity(partition, coinHoardGraph)

colors = [partition[n] for n in coinHoardGraph.nodes()]
my_colors = plt.cm.Set2 # you can select other color pallettes here: https://matplotlib.org/users/colormaps.html

dataframeResults = {}
dataframeResults = createMapDfs(dfEgoHoard, partition, coinHoardGraph, my_colors)


hoardsinputdf = dataframeResults['hoardsDf'].copy()
hoardsinputdf['color'] = hoardsinputdf.apply (lambda row: makeNetColor(row, my_colors), axis=1)

hoardsdf = pd.DataFrame()
hoardsfg = folium.FeatureGroup(name='Coin Hoards Network: {mint}'.format(mint = 'myrina' ))
addFeatureGrouptoMap(hoardsinputdf, hoardsfg, 30, 5, 'degree', 'title', m)

gdfmyrina = polityRouteMaker (dfEgoHoard, 'location_uri', 'aa_locations', 'places', mint["myrina"])



In [ ]:
folium.Choropleth(
    gdfmyrina,
    line_weight=3,
    line_color='orange'
).add_to(m)

In [ ]:
start_date = -153
end_date = -145
buffer = 2

m = createmapvis()

dfEgoHoard = pd.DataFrame()
dfEgoHoard = hoardsEgoDisplay (mint["kyme"], start_date, end_date, buffer)
dfEgoHoardEdge = dfEgoHoard.groupby(['mint_uri', 'hoard_id',]).sum().reset_index()

coinHoardGraph = nx.from_pandas_edgelist(dfEgoHoardEdge, 'mint_uri', 'hoard_id','count')
coinHoardGraphDict = dict(coinHoardGraph.degree)
partition = community.best_partition(coinHoardGraph)
modularity = community.modularity(partition, coinHoardGraph)

colors = [partition[n] for n in coinHoardGraph.nodes()]
my_colors = plt.cm.Set2 # you can select other color pallettes here: https://matplotlib.org/users/colormaps.html

dataframeResults = {}
dataframeResults = createMapDfs(dfEgoHoard, partition, coinHoardGraph, my_colors)


hoardsinputdf = dataframeResults['hoardsDf'].copy()
hoardsinputdf['color'] = hoardsinputdf.apply (lambda row: makeNetColor(row, my_colors), axis=1)

hoardsdf = pd.DataFrame()
hoardsfg = folium.FeatureGroup(name='Coin Hoards Network: {mint}'.format(mint = 'kyme' ))
addFeatureGrouptoMap(hoardsinputdf, hoardsfg, 30, 5, 'degree', 'title', m)

gdfkyme = polityRouteMaker (dfEgoHoard, 'location_uri', 'aa_locations', 'places', mint["kyme"])



In [ ]:
folium.Choropleth(
    gdfkyme,
    line_weight=3,
    line_color='yellow'
).add_to(m)

In [ ]:
polity = 550812

dfAttalid = ancientEmpireRoutes(polity, 'aa_hellenistic_edges', -153, -145)
dfAttalidPoints = ancientEmpirePoints(polity, 'aa_hellenistic_edges', -153, -145)

dfAttalidRoutes = gpd.GeoDataFrame = polityRouteMaker (dfAttalid, 'id', 'places', 'places', polity)


In [ ]:
polity = 45635759

dfSeleukid = ancientEmpireRoutes(polity, 'aa_hellenistic_edges', -153, -145)
dfSeleukidPoints = ancientEmpirePoints(polity, 'aa_hellenistic_edges', -153, -145)

dfSeleukidRoutes = gpd.GeoDataFrame = polityRouteMaker (dfSeleukid, 'id', 'places', 'places', polity)


In [ ]:

folium.Choropleth(
    dfAttalidRoutes,
    line_weight=3,
    line_color='red'
).add_to(m)

folium.Choropleth(
    dfSeleukidRoutes,
    line_weight=3,
    line_color='blue'
).add_to(m)

m

While the data behind this map can be extensively explored in jupyter, it can also be exported to QGIS and styled there (and touched up in inkscape).

In [ ]:
gdfTemnos.to_file('temnos_routes', driver="GeoJSON")
gdfmyrina.to_file('myrina_routes', driver="GeoJSON")
gdfkyme.to_file('kyme_routes', driver="GeoJSON")

dfAttalidRoutes.to_file('attalid_routes', driver="GeoJSON")
dfSeleukidRoutes.to_file('seleukid_routes', driver="GeoJSON")


 <img src="img/balas_mints.png" alt="Networks surrounding Alexander Balas" style="width: 1000px;"/>

In [ ]:
import sqlite3
conn = sqlite3.connect("https://github.com/Aeolian-Alexanders/data/blob/master/sqlite%20database/aeolian_alexanders.db?raw=true")


In [ ]:
cur = conn.cursor()
cur.execute("select * from all_coins limit 5;")
